<a href="https://colab.research.google.com/github/vijaycma/public/blob/main/BifFig_Case_Slides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install --upgrade openpyxl
import openpyxl # to open Excel files
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame 

!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum

#For maps:
!pip install gmaps
!pip install widgetsnbextension
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html
import IPython
import gmaps # Doc: https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#basic-concepts

%matplotlib inline

     |████████████████████████████████| 11.5 MB 12.6 MB/s 
     |████████████████████████████████| 1.1 MB 14.6 MB/s 
  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=a4cb28684520ac1f528154213c040f22ab2ada6351f64cab83e84411de26b745
  Stored in directory: /root/.cache/pip/wheels/3d/1c/cb/c46f4199210e632b28d78d05a00bfdccbf6dfcd3f5b051d6bf
Successfully built gmaps


In [ ]:
# Read Excel File from Fabio's dropbox
!wget -O BigFig_Data_To_Post.xlsx https://www.dropbox.com/s/ym1n9h0txm0bz9k/15_763_Class05_BigFig_Data_To_Post.xlsx?dl=0

--2022-04-10 22:10:32--  https://www.dropbox.com/s/ym1n9h0txm0bz9k/15_763_Class05_BigFig_Data_To_Post.xlsx?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:601a:18::a27d:712
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ym1n9h0txm0bz9k/15_763_Class05_BigFig_Data_To_Post.xlsx [following]
--2022-04-10 22:10:32--  https://www.dropbox.com/s/raw/ym1n9h0txm0bz9k/15_763_Class05_BigFig_Data_To_Post.xlsx
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2233900bb0c50c3d956b29c788.dl.dropboxusercontent.com/cd/0/inline/BjI5xBQ47I55S3QSTswDpyPHW-k2NOuioGfdS7n3DleHjA9DHMd4lHoBRhVZ1uxjw60g5AmA4TuL_p8gVrbds11I6eaB2gReKp0xYms6Dj3vokki4HXAUfyfg5oYSVINrvIweyWxp80yn3QtpZ7r2mH7qEUzwUWM1kaMt1PyAfJabQ/file# [following]
--2022-04-10 22:10:33--  https://uc2233900bb0c50c3d956b29c788.dl.dropboxusercont

In [ ]:
#Read data
demand_data = pd.read_excel('BigFig_Data_To_Post.xlsx', sheet_name='Demand data', engine='openpyxl')
plant_locations = pd.read_excel('BigFig_Data_To_Post.xlsx', sheet_name='Plant locations', engine='openpyxl')
plant_distances = pd.read_excel('BigFig_Data_To_Post.xlsx', sheet_name='Plant distance', engine='openpyxl')
customer_distances = pd.read_excel('BigFig_Data_To_Post.xlsx', sheet_name='Plant_to_Customer_Distance', engine='openpyxl')


In [ ]:
print(list(demand_data))
demand_data.head(4)

['City', 'State', 'Latitude', 'Longitude', 'Assigned plant', '2017 case demand', '2019 case demand']


,City,State,Latitude,Longitude,Assigned plant,2017 case demand,2019 case demand
0,New York,New York,40.7128,-74.0059,York,18069,20048
1,Los Angeles,California,34.0522,-118.2437,Stockton,12046,14541
2,Houston,Texas,29.7604,-95.3698,Rockwall,0,11530
3,Phoenix,Arizona,33.4484,-112.0740,Stockton,9637,11702


In [ ]:
print(list(plant_locations))
plant_locations.head(6)

['Plant Type', 'City', 'State', 'Latitude', 'Longitude']


,Plant Type,City,State,Latitude,Longitude
0,Fig processing,Acampo,California,38.1746,-121.2786
1,Bottling plant,Stockton,California,37.9577,-121.2908
2,Bottling plant,Rockwall,Texas,32.9312,-96.4597
3,Bottling plant,Joliet,Illinois,41.5250,-88.0817
4,Bottling plant,Atlanta,Georgia,33.7490,-84.3880
5,Bottling plant,York,Pennsylvania,39.9626,-76.7277


In [ ]:
print(list(plant_distances))
plant_distances.head(4)

['Unnamed: 0', 'Acampo, CA', 'Stockton, CA', 'Rockwall, TX', 'Joliet, IL', 'Atlanta, GA', 'York, PA']


,Unnamed: 0,"Acampo, CA","Stockton, CA","Rockwall, TX","Joliet, IL","Atlanta, GA","York, PA"
0,"Acampo, CA",0.0,18.9,1710,2050,2436,2736
1,"Stockton, CA",18.9,0.0,1698,2071,2423,2757
2,"Rockwall, TX",1710.0,1698.0,0,902,770,1359
3,"Joliet, IL",2050.0,2071.0,902,0,732,693


In [ ]:
print(list(customer_distances))
customer_distances.head(4)

['Customer Location', 'Latitude,Longitude', 'Latitude', 'Longitude', 'Stockton, CA', 'Rockwall, TX', 'Joliet, IL', 'Atlanta, GA', 'York, PA']


,Customer Location,"Latitude,Longitude",Latitude,Longitude,"Stockton, CA","Rockwall, TX","Joliet, IL","Atlanta, GA","York, PA"
0,"New York,New York","40.7128,-74.0059",40.7128,-74.0059,2878,1527,808,862,185
1,"Los Angeles,California","34.0522,-118.2437",34.0522,-118.2437,339,1460,1992,2175,2633
2,"Houston,Texas","29.7604,-95.3698",29.7604,-95.3698,1885,258,1039,793,1482
3,"Phoenix,Arizona","33.4484,-112.074",33.4484,-112.0740,710,1088,1723,1846,2265


In [ ]:
'1'+','

'1,'

In [ ]:
demand_data['Customer Location'] = demand_data['City']+','+demand_data['State']
demand_data.head()

,City,State,Latitude,Longitude,Assigned plant,2017 case demand,2019 case demand,Customer Location
0,New York,New York,40.7128,-74.0059,York,18069,20048,"New York,New York"
1,Los Angeles,California,34.0522,-118.2437,Stockton,12046,14541,"Los Angeles,California"
2,Houston,Texas,29.7604,-95.3698,Rockwall,0,11530,"Houston,Texas"
3,Phoenix,Arizona,33.4484,-112.0740,Stockton,9637,11702,"Phoenix,Arizona"
4,San Antonio,Texas,29.4241,-98.4936,Rockwall,0,7744,"San Antonio,Texas"


In [ ]:
customer_distances = customer_distances.merge(demand_data[['Customer Location','2017 case demand']], how='left', on='Customer Location')
customer_distances.head()

,Customer Location,"Latitude,Longitude",Latitude,Longitude,"Stockton, CA","Rockwall, TX","Joliet, IL","Atlanta, GA","York, PA",2017 case demand
0,"New York,New York","40.7128,-74.0059",40.7128,-74.0059,2878,1527,808,862,185,18069
1,"Los Angeles,California","34.0522,-118.2437",34.0522,-118.2437,339,1460,1992,2175,2633,12046
2,"Houston,Texas","29.7604,-95.3698",29.7604,-95.3698,1885,258,1039,793,1482,0
3,"Phoenix,Arizona","33.4484,-112.074",33.4484,-112.0740,710,1088,1723,1846,2265,9637
4,"San Antonio,Texas","29.4241,-98.4936",29.4241,-98.4936,1689,298,1169,988,1677,0


In [ ]:
#Model data and parameters

n_plants = 5
n_customers = len(customer_distances)

# customer distances from data
distances_customers = customer_distances[['Stockton, CA', 'Rockwall, TX', 'Joliet, IL', 'Atlanta, GA', 'York, PA']].to_numpy()
distances_customers = np.transpose(distances_customers)
# plant distances from data
distances_plants = plant_distances['Acampo, CA'].to_numpy()
distances_plants = distances_plants[1:n_customers]
# Demand from data
demand = demand_data['2017 case demand'].to_numpy()

#Chelsey changed b_i and c_ij to include cases/truck in the denominator (distances are already in the objective funciton below)
b = [1/7000 for i in range(n_plants)] 
c = [[1/1296 for j in range(n_customers)] for i in range(n_plants)] 
#capacity = [2000000]*5 # CORRECT WITH PROBLEM INFORMATION
capacity = ([250000, 15000, 175000, 150000, 200000]) 


max_plants = 5 #Chelsey changed from 3 to 5 once capacity data was updated 

In [ ]:
# Initialize new Gurobi Model
m = gp.Model("BigFig")

# Create variables: 
# variable x = route from Acampo to plants i
X = dict( [((i), m.addVar(name="x_%s" %(i))) for i in range(n_plants) ] )

# variable open = if plant i is open
open_plant = dict( [((i), m.addVar(vtype=GRB.BINARY, name="open_%s" %(i))) for i in range(n_plants) ] )

# variable y = plant i to customer j
Y = dict( [((i,j), m.addVar(name="y_%s_%s" %(i,j))) for i in range(n_plants) for j in range(n_customers)] )



Restricted license - for non-production use only - expires 2023-10-25


In [ ]:

# Set objective: Minimize 􏰍bixi +􏰍􏰍cijyij
m.setObjective(
    quicksum([b[i] * distances_plants[i]*X[(i)] for i in range(n_plants)]) + quicksum([c[i][j] * distances_customers[i][j]*Y[(i,j)] for i in range(n_plants) for j in range(n_customers)]), 
    GRB.MINIMIZE)

# Add constraints:
# The first constraint forces the total cases shipped to customer location j to be no less than its average weekly demand
# 􏰍 yij ≥ dj
for j in range(n_customers):
  m.addConstr(quicksum([Y[(i,j)] for i in range(n_plants) ]) >= demand[j])

# The second constraint restricts a bottling plant to ship no more than it receives weekly from Acampo.
# 􏰍yij ≤xi
for i in range(n_plants):
  m.addConstr(quicksum([Y[(i,j)] for j in range(n_customers) ]) <= X[i])

# The third constraint does not let bottling plant i ship out more than its weekly case capacity.
# 􏰍yij ≤δiei
for i in range(n_plants):
  m.addConstr(quicksum([Y[(i,j)] for j in range(n_customers) ]) <= open_plant[i] * capacity[i])

# The fourth constraint limits the total number of bottling plants to N.
# 􏰍 δi ≤ N
m.addConstr(quicksum([open_plant[i] for i in range(n_plants) ]) <= max_plants)

# The remaining constraints impose non-negativity on the shipping quantities
# xi ≥ 0
# yij ≥0
for i in range(n_plants):
  m.addConstr(X[i] >= 0)
  for j in range(n_customers):
    m.addConstr(Y[(i,j)] >= 0)

m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads
Optimize a model with 1803 rows, 750 columns and 5215 nonzeros
Model fingerprint: 0x82efa3d8
Variable types: 745 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+05]
  Objective range  [3e-03, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 2e+04]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 1658 rows and 50 columns
Presolve time: 0.01s
Presolved: 145 rows, 700 columns, 1400 nonzeros
Variable types: 700 continuous, 0 integer (0 binary)

Root relaxation: objective 3.545159e+05, 146 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    354515.88949 354515.889  0.00%     -    0s

Explore

In [ ]:
print(X)
print(open_plant)
print(Y)

{0: <gurobi.Var x_0 (value 131305.0)>, 1: <gurobi.Var x_1 (value 51799.0)>, 2: <gurobi.Var x_2 (value 130101.0)>, 3: <gurobi.Var x_3 (value 128894.0)>, 4: <gurobi.Var x_4 (value 212010.0)>}
{0: <gurobi.Var open_0 (value 0.0)>, 1: <gurobi.Var open_1 (value 0.0)>, 2: <gurobi.Var open_2 (value 0.0)>, 3: <gurobi.Var open_3 (value 0.0)>, 4: <gurobi.Var open_4 (value 0.0)>}
{(0, 0): <gurobi.Var y_0_0 (value 0.0)>, (0, 1): <gurobi.Var y_0_1 (value 12046.0)>, (0, 2): <gurobi.Var y_0_2 (value 0.0)>, (0, 3): <gurobi.Var y_0_3 (value 9637.0)>, (0, 4): <gurobi.Var y_0_4 (value 0.0)>, (0, 5): <gurobi.Var y_0_5 (value 1205.0)>, (0, 6): <gurobi.Var y_0_6 (value 0.0)>, (0, 7): <gurobi.Var y_0_7 (value 9637.0)>, (0, 8): <gurobi.Var y_0_8 (value 0.0)>, (0, 9): <gurobi.Var y_0_9 (value 0.0)>, (0, 10): <gurobi.Var y_0_10 (value 0.0)>, (0, 11): <gurobi.Var y_0_11 (value 0.0)>, (0, 12): <gurobi.Var y_0_12 (value 0.0)>, (0, 13): <gurobi.Var y_0_13 (value 12046.0)>, (0, 14): <gurobi.Var y_0_14 (value 0.0)>, (

In [ ]:
Y[(0,1)].x

12046.0

In [ ]:
for i in range(n_plants):
  for j in range(n_customers):
      print(Y[(i,j)].x)
  

0.0
12046.0
0.0
9637.0
0.0
1205.0
0.0
9637.0
0.0
0.0
0.0
0.0
0.0
12046.0
0.0
0.0
0.0
0.0
12046.0
9637.0
0.0
0.0
2409.0
4818.0
0.0
0.0
0.0
12046.0
0.0
0.0
0.0
0.0
1205.0
0.0
1205.0
1205.0
1205.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1205.0
7228.0
0.0
7228.0
0.0
1205.0
0.0
0.0
0.0
0.0
0.0
4818.0
0.0
0.0
0.0
0.0
0.0
0.0
1205.0
0.0
0.0
4818.0
1205.0
0.0
0.0
0.0
0.0
0.0
4818.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1205.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
2409.0
0.0
0.0
1205.0
2409.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
12046.0
0.0
0.0
0.0
0.0
0.0
1205.0
0.0
9637.0
0.0
0.0
2409.0
0.0
0.0
0.0
1205.0
0.0
0.0
2409.0
0.0
0.0
0.0
0.0
2409.0
2409.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1205.0
0.0
0.0
0.0
2409.0
0.0


In [ ]:
customer_distances2 = customer_distances[customer_distances['2017 case demand']>0]
n_customers2 = len(customer_distances2)

In [ ]:
customer_distances

,Customer Location,"Latitude,Longitude",Latitude,Longitude,"Stockton, CA","Rockwall, TX","Joliet, IL","Atlanta, GA","York, PA",2017 case demand
0,"New York,New York","40.7128,-74.0059",40.7128,-74.0059,2878,1527,808,862,185,18069
1,"Los Angeles,California","34.0522,-118.2437",34.0522,-118.2437,339,1460,1992,2175,2633,12046
2,"Houston,Texas","29.7604,-95.3698",29.7604,-95.3698,1885,258,1039,793,1482,0
3,"Phoenix,Arizona","33.4484,-112.074",33.4484,-112.0740,710,1088,1723,1846,2265,9637
4,"San Antonio,Texas","29.4241,-98.4936",29.4241,-98.4936,1689,298,1169,988,1677,0
...,...,...,...,...,...,...,...,...,...,...
143,"Woonsocket,Rhode Island","42.0029,-71.5148",42.0029,-71.5148,3045,1722,976,1057,380,4818
144,"Mankato,Minnesota","44.1636,-93.9994",44.1636,-93.9994,1851,895,454,1076,1106,1205
145,"Bentonville,Arkansas","36.3729,-94.2088",36.3729,-94.2088,1794,348,599,726,1141,1205
146,"Marlborough,Massachusetts","42.3459,-71.5523",42.3459,-71.5523,3042,1718,973,1053,376,4818


In [ ]:
# Load Google Maps API Key and create figure
gmaps.configure(api_key="PUT YOUR GOOGLE API KEY HERE")

fig = gmaps.figure()

def draw_square(lat, lon, size,color):
  return gmaps.Polygon([(lat+size/3, lon+size/2), (lat-size/3, lon+size/2), (lat-size/3, lon-size/2), (lat+size/3, lon-size/2)],fill_color=color,stroke_color='black',fill_opacity = 1)

latitude_source = plant_locations['Latitude'].to_numpy()[0]
longitude_source = plant_locations['Longitude'].to_numpy()[0]
latitude_plants = plant_locations['Latitude'].to_numpy()[1:n_plants+1]
longitude_plants =  plant_locations['Longitude'].to_numpy()[1:n_plants+1]
latitude_customers = customer_distances['Latitude'].to_numpy()
longitude_customers = customer_distances['Longitude'].to_numpy()
latitude_customers2 = customer_distances2['Latitude'].to_numpy()
longitude_customers2 = customer_distances2['Longitude'].to_numpy()

# Draw Routes
linewidth = 0.0001

# Plants to customers
map_lines = []
for i in range(n_plants):
  if X[i].x > 0:
    map_lines.append(gmaps.Line(start=(latitude_plants[i], longitude_plants[i]),end=(latitude_source, longitude_source),stroke_color = (59,196,227), stroke_weight = 5))
  for j in range(len(customer_distances)):
    if Y[(i,j)].x > 0:
      map_lines.append(gmaps.Line(start=(latitude_plants[i], longitude_plants[i]),end=(latitude_customers[j], longitude_customers[j]),stroke_color = (250,0,0), stroke_weight = 3))
drawing = gmaps.drawing_layer(features=map_lines)
fig.add_layer(drawing)

# Add markers to plants and customers
map_circles = []
map_circles.append(draw_square(latitude_source, longitude_source, 1.2,''))
for i in range(n_plants):
  map_circles.append(draw_square(latitude_plants[i], longitude_plants[i], 0.75,'blue'))
for j in range(n_customers2):
  map_circles.append(draw_square(latitude_customers2[j], longitude_customers2[j], 0.5,'#A52A2A'))

drawing = gmaps.drawing_layer(features = map_circles)
#print(drawing)
fig.add_layer(drawing)

gmaps.figure(map_type='SATELLITE') #'ROADMAP', 'SATELLITE', 'HYBRID' or 'TERRAIN' 

fig
embed_minimal_html('export.html', views=[fig])
IPython.display.HTML(filename="export.html")


In [ ]:
demand_data2 = demand_data.copy()
demand_data2 = demand_data2.merge(plant_locations[['City','Latitude','Longitude']],how='left', left_on='Assigned plant', right_on='City')
demand_data2 = demand_data2[demand_data2['2017 case demand']>0]


In [ ]:
plant_lat = demand_data2.Latitude_y.to_numpy()
plant_lon = demand_data2.Longitude_y.to_numpy()
cust_lat = demand_data2.Latitude_x.to_numpy()
cust_lon = demand_data2.Longitude_x.to_numpy()

In [ ]:
# Load Google Maps API Key and create figure
gmaps.configure(api_key="PUT YOUR GOOGLE API KEY HERE")

fig = gmaps.figure()

def draw_square(lat, lon, size,color):
  return gmaps.Polygon([(lat+size/3, lon+size/2), (lat-size/3, lon+size/2), (lat-size/3, lon-size/2), (lat+size/3, lon-size/2)],fill_color=color,stroke_color='black',fill_opacity = 1)

latitude_source = plant_locations['Latitude'].to_numpy()[0]
longitude_source = plant_locations['Longitude'].to_numpy()[0]
latitude_plants = plant_locations['Latitude'].to_numpy()[1:n_plants+1]
longitude_plants =  plant_locations['Longitude'].to_numpy()[1:n_plants+1]
latitude_customers = customer_distances['Latitude'].to_numpy()
longitude_customers = customer_distances['Longitude'].to_numpy()
latitude_customers2 = customer_distances2['Latitude'].to_numpy()
longitude_customers2 = customer_distances2['Longitude'].to_numpy()

# Draw Routes
linewidth = 0.0001

# Plants to customers
map_lines = []
for i in range(n_plants):
  if X[i].x > 0:
    map_lines.append(gmaps.Line(start=(latitude_plants[i], longitude_plants[i]),end=(latitude_source, longitude_source),stroke_color = (59,196,227), stroke_weight = 5))
  # for j in range(len(customer_distances)):
  #   if Y[(i,j)].x > 0:
  #     map_lines.append(gmaps.Line(start=(latitude_plants[i], longitude_plants[i]),end=(latitude_customers[j], longitude_customers[j]),stroke_color = (250,0,0), stroke_weight = 3))

for j in range(len(cust_lat)):
  map_lines.append(gmaps.Line(start=(plant_lat[j], plant_lon[j]),end=(cust_lat[j], cust_lon[j]),stroke_color = (250,0,0), stroke_weight = 3))
drawing = gmaps.drawing_layer(features=map_lines)
fig.add_layer(drawing)

# Add markers to plants and customers
map_circles = []
map_circles.append(draw_square(latitude_source, longitude_source, 1.2,'green'))
for i in range(n_plants):
  map_circles.append(draw_square(latitude_plants[i], longitude_plants[i], 0.75,'blue'))
for j in range(n_customers2):
  map_circles.append(draw_square(latitude_customers2[j], longitude_customers2[j], 0.5,'#A52A2A'))

drawing = gmaps.drawing_layer(features = map_circles)
#print(drawing)
fig.add_layer(drawing)

gmaps.figure(map_type='SATELLITE') #'ROADMAP', 'SATELLITE', 'HYBRID' or 'TERRAIN' 

fig
embed_minimal_html('export.html', views=[fig])
IPython.display.HTML(filename="export.html")


In [ ]:
n_customers2

140

In [ ]:
customer_distances

,Customer Location,"Latitude,Longitude",Latitude,Longitude,"Stockton, CA","Rockwall, TX","Joliet, IL","Atlanta, GA","York, PA",2017 case demand
0,"New York,New York","40.7128,-74.0059",40.7128,-74.0059,2878,1527,808,862,185,18069
1,"Los Angeles,California","34.0522,-118.2437",34.0522,-118.2437,339,1460,1992,2175,2633,12046
2,"Houston,Texas","29.7604,-95.3698",29.7604,-95.3698,1885,258,1039,793,1482,0
3,"Phoenix,Arizona","33.4484,-112.074",33.4484,-112.0740,710,1088,1723,1846,2265,9637
4,"San Antonio,Texas","29.4241,-98.4936",29.4241,-98.4936,1689,298,1169,988,1677,0
...,...,...,...,...,...,...,...,...,...,...
143,"Woonsocket,Rhode Island","42.0029,-71.5148",42.0029,-71.5148,3045,1722,976,1057,380,4818
144,"Mankato,Minnesota","44.1636,-93.9994",44.1636,-93.9994,1851,895,454,1076,1106,1205
145,"Bentonville,Arkansas","36.3729,-94.2088",36.3729,-94.2088,1794,348,599,726,1141,1205
146,"Marlborough,Massachusetts","42.3459,-71.5523",42.3459,-71.5523,3042,1718,973,1053,376,4818


In [ ]:
latitude_plants

array([37.9577, 32.9312, 41.525 , 33.749 , 39.9626])